# Low-rank approximation on $\mathcal{P}(d)$ - the space of $d$-dimensional SPD matrices

In this notebook we want to get some intuition in different approaches for computing low-rank approximations for manifold-valued signals

In [1]:
using Manifolds
using Manopt
using LinearAlgebra
using Random
using Plots
using LaTeXStrings
using BenchmarkTools

In [2]:
include("../../../src/decompositions/signals/naive_low_rank_approximation.jl")
include("../../../src/decompositions/signals/curvature_corrected_low_rank_approximation.jl")
include("../../../src/decompositions/signals/exact_low_rank_approximation.jl")
include("../../../src/decompositions/signals/stochastic_curvature_corrected_low_rank_approximation.jl")

include("../../../src/functions/loss_functions/curvature_corrected_loss.jl")
include("../../../src/functions/loss_functions/exact_loss.jl")
include("../../../src/functions/jacobi_field/beta.jl")

β (generic function with 1 method)

In [3]:
black = RGBA{Float64}(colorant"#000000")
TolVibrantOrange = RGBA{Float64}(colorant"#EE7733")
TolVibrantBlue = RGBA{Float64}(colorant"#0077BB")
TolVibrantTeal = RGBA{Float64}(colorant"#009988")
TolVibrantMagenta = RGBA{Float64}(colorant"#EE3377")
TolVibrantCyan = RGBA{Float64}(colorant"#33BBEE");
render_size = 1

1

### Load data and construct manifold ###

In [4]:
# load data
M = Sphere(6)
d = manifold_dimension(M)
n = 100  # 100


100

In [5]:
θ = collect(range(0, 2 * π, n+1))[1:n]
predata = [[cos(θ[i]), sin(θ[i]), 0., 0., 0., 0., 0.] for i in 1:n]
σ = .05  # variance

Random.seed!(31)
data = [exp(M, predata[i], random_tangent(M, predata[i], Val(:Gaussian), σ)) for i in 1:n]; # ∈ S(6)^n

In [6]:
# Export slice image
# asymptote_export_S2_signals("results/artificial1D_orig.asy", points=[data],
# colors=Dict(:points => [TolVibrantBlue]),
# dot_size=3.5,
# camera_position=(1.0, 0.5, 0.5)); 

### Construct low rank approximation ###

In [7]:
q₁ = mean(M, data)
println(q₁)
log_q_data₁ = log.(Ref(M), Ref(q₁), data);  # ∈ T_q P(3)^n
ONB₁ = get_basis.(Ref(M), Ref(q₁), DiagonalizingOrthonormalBasis.(log_q_data₁))
κ₁ = maximum([maximum(distance(M, q₁, data[i])^2 .* ONB₁[i].data.eigenvalues) for i in 1:n])
println(κ₁)


summed_pairwise_distances = [sum([distance.(Ref(M), Ref(data[i]), data) .^2]) for i in 1:n]
q₂ =  data[argmin(summed_pairwise_distances)]
log_q_data₂ = log.(Ref(M), Ref(q₂), data);  # ∈ T_q P(3)^n
println(q₂)
ONB₂ = get_basis.(Ref(M), Ref(q₂), DiagonalizingOrthonormalBasis.(log_q_data₂))
κ₂ = maximum([maximum(distance(M, q₂, data[i])^2 .* ONB₂[i].data.eigenvalues) for i in 1:n])
println(κ₂)

[0.011362771023773281, -0.014665511087507247, 0.32104150872364784, -0.4013489917107375, -0.07156976421128587, -0.852341865783546, -0.06243604273002383]
2.870971878384671
[0.9944358300450549, -0.05035588364543773, 0.05181675794418579, 0.03470388913095954, 0.013728091849230824, 0.009286850030275141, -0.06631457223660438]
9.16012544722712


In [8]:
exact_low_rank_approximation(M, q₁, data, 1; stepsize=1/64, max_iter=500, print_iterates=true);

Initial  F(x): 0.40744920354 | 
# 1     change: 0.070133459 |  F(x): 0.40640345319 | 
# 2     change: 0.068347618 |  F(x): 0.40541838533 | 
# 3     change: 0.066607254 |  F(x): 0.40449074930 | 
# 4     change: 0.064911216 |  F(x): 0.40361746224 | 
# 5     change: 0.063258380 |  F(x): 0.40279560058 | 
# 6     change: 0.061647653 |  F(x): 0.40202239204 | 
# 7     change: 0.060077964 |  F(x): 0.40129520794 | 
# 8     change: 0.058548274 |  F(x): 0.40061155597 | 
# 9     change: 0.057057566 |  F(x): 0.39996907327 | 
# 10    change: 0.055604850 |  F(x): 0.39936551985 | 
# 11    change: 0.054189160 |  F(x): 0.39879877237 | 
# 12    change: 0.052809555 |  F(x): 0.39826681818 | 
# 13    change: 0.051465116 |  F(x): 0.39776774973 | 
# 14    change: 0.050154949 |  F(x): 0.39729975919 | 
# 15    change: 0.048878180 |  F(x): 0.39686113334 | 
# 16    change: 0.047633961 |  F(x): 0.39645024880 | 
# 17    change: 0.046421461 |  F(x): 0.39606556739 | 
# 18    change: 0.045239872 |  F(x): 0.39570563180

In [9]:
exact_low_rank_approximation(M, q₂, data, 2; stepsize=1/16, max_iter=1000, print_iterates=true); 

Initial  F(x): 0.00672018240 | 
# 1     change: 0.086798872 |  F(x): 0.00635742018 | 
# 2     change: 0.078021022 |  F(x): 0.00606489683 | 
# 3     change: 0.070304808 |  F(x): 0.00582787993 | 
# 4     change: 0.063521004 |  F(x): 0.00563483879 | 
# 5     change: 0.057555336 |  F(x): 0.00547673833 | 
# 6     change: 0.052306753 |  F(x): 0.00534648962 | 
# 7     change: 0.047685925 |  F(x): 0.00523852223 | 
# 8     change: 0.043613928 |  F(x): 0.00514845139 | 
# 9     change: 0.040021109 |  F(x): 0.00507281868 | 
# 10    change: 0.036846082 |  F(x): 0.00500889019 | 
# 11    change: 0.034034863 |  F(x): 0.00495449913 | 
# 12    change: 0.031540081 |  F(x): 0.00490792317 | 
# 13    change: 0.029320294 |  F(x): 0.00486778880 | 
# 14    change: 0.027339350 |  F(x): 0.00483299666 | 
# 15    change: 0.025565824 |  F(x): 0.00480266321 | 
# 16    change: 0.023972485 |  F(x): 0.00477607525 | 
# 17    change: 0.022535816 |  F(x): 0.00475265419 | 
# 18    change: 0.021235571 |  F(x): 0.00473192825

In [10]:
max_iter = 1000

nR_q₁ = []
nU₁ = []
ccR_q₁ = []
ccU₁ = []
eR_q₁ = []
eU₁ = []
eCosts₁ = []

nR_q₂ = []
nU₂ = []
ccR_q₂ = []
ccU₂ = []
eR_q₂ = []
eU₂ = []
eCosts₂ = []

for i in 1:d
    println("#$(i) | computing naive low-rank approximation")
    nRr_q₁, nUr₁ = naive_low_rank_approximation(M, q₁, data, i)
    push!(nR_q₁, nRr_q₁)
    push!(nU₁, nUr₁)
    println("#$(i) | computing curvature corrected low-rank approximation")
    ccRr_q₁, ccUr₁ = curvature_corrected_low_rank_approximation(M, q₁, data, i); 
    push!(ccR_q₁, ccRr_q₁)
    push!(ccU₁, ccUr₁)
    println("#$(i) | computing exact low-rank approximation")
    (eRr_q₁, eUr₁), eCostsr₁ = exact_low_rank_approximation(M, q₁, data, i; stepsize=1/64, max_iter=max_iter); 
    push!(eR_q₁, eRr_q₁)
    push!(eU₁, eUr₁)
    push!(eCosts₁, eCostsr₁)

    println("#$(i) | computing naive low-rank approximation")
    nRr_q₂, nUr₂ = naive_low_rank_approximation(M, q₂, data, i)
    push!(nR_q₂, nRr_q₂)
    push!(nU₂, nUr₂)
    println("#$(i) | computing curvature corrected low-rank approximation")
    ccRr_q₂, ccUr₂ = curvature_corrected_low_rank_approximation(M, q₂, data, i); 
    push!(ccR_q₂, ccRr_q₂)
    push!(ccU₂, ccUr₂)
    println("#$(i) | computing exact low-rank approximation")
    (eRr_q₂, eUr₂), eCostsr₂ = exact_low_rank_approximation(M, q₂, data, i; stepsize=1/16, max_iter=max_iter); 
    push!(eR_q₂, eRr_q₂)
    push!(eU₂, eUr₂)
    push!(eCosts₂, eCostsr₂)
end

#1 | computing naive low-rank approximation
#1 | computing curvature corrected low-rank approximation
#1 | computing exact low-rank approximation
#1 | computing naive low-rank approximation
#1 | computing curvature corrected low-rank approximation
#1 | computing exact low-rank approximation
#2 | computing naive low-rank approximation
#2 | computing curvature corrected low-rank approximation
#2 | computing exact low-rank approximation
#2 | computing naive low-rank approximation
#2 | computing curvature corrected low-rank approximation
#2 | computing exact low-rank approximation
#3 | computing naive low-rank approximation
#3 | computing curvature corrected low-rank approximation
#3 | computing exact low-rank approximation
#3 | computing naive low-rank approximation
#3 | computing curvature corrected low-rank approximation
#3 | computing exact low-rank approximation
#4 | computing naive low-rank approximation
#4 | computing curvature corrected low-rank approximation
#4 | computing exact l

In [11]:
# basepoint q₁
ref_distance₁ = sum(distance.(Ref(M), Ref(q₁), data).^2)

naive_tangent_distances_r₁ = zeros(d)
predicted_naive_distances_r₁ = zeros(d)
true_naive_distances_r₁ = zeros(d)

curvature_corrected_tangent_distances_r₁ = zeros(d)
predicted_curvature_corrected_distances_r₁ = zeros(d)
true_curvature_corrected_distances_r₁ = zeros(d)

exact_tangent_distances_r₁ = zeros(d)
exact_distances_r₁ = zeros(d)

# basepoint q₂
ref_distance₂ = sum(distance.(Ref(M), Ref(q₂), data).^2)

naive_tangent_distances_r₂ = zeros(d)
predicted_naive_distances_r₂ = zeros(d)
true_naive_distances_r₂ = zeros(d)

curvature_corrected_tangent_distances_r₂ = zeros(d)
predicted_curvature_corrected_distances_r₂ = zeros(d)
true_curvature_corrected_distances_r₂ = zeros(d)

exact_tangent_distances_r₂ = zeros(d)
exact_distances_r₂ = zeros(d)

for rank in 1:d
    ## basepoint q₁
    naive_log_q_data_r₁ = [sum([nR_q₁[rank][i] * nU₁[rank][k,i] for i in 1:rank]) for k in 1:n]
    curvature_corrected_log_q_data_r₁ = [sum([ccR_q₁[rank][i] * ccU₁[rank][k,i] for i in 1:rank]) for k in 1:n]
    exact_log_q_data_r₁ = [sum([eR_q₁[rank][i] * eU₁[rank][k,i] for i in 1:rank]) for k in 1:n]
    
    # expoentiate back
    naive_data_r₁ = exp.(Ref(M), Ref(q₁), naive_log_q_data_r₁)
    curvature_corrected_data_r₁ = exp.(Ref(M), Ref(q₁), curvature_corrected_log_q_data_r₁)
    exact_data_r₁ = exp.(Ref(M), Ref(q₁), exact_log_q_data_r₁)


    # compute relative tangent space error
    naive_tangent_distances_r₁[rank] = sum(norm.(Ref(M), Ref(q₁),  log_q_data₁ - naive_log_q_data_r₁).^2) / ref_distance₁
    curvature_corrected_tangent_distances_r₁[rank] = sum(norm.(Ref(M), Ref(q₁),  log_q_data₁ - curvature_corrected_log_q_data_r₁).^2) / ref_distance₁
    exact_tangent_distances_r₁[rank] = sum(norm.(Ref(M), Ref(q₁),  log_q_data₁ - exact_log_q_data_r₁).^2) / ref_distance₁


    # compute relative manifold error
    predicted_naive_distances_r₁[rank] = curvature_corrected_loss(M, q₁, data, naive_log_q_data_r₁)
    true_naive_distances_r₁[rank] = exact_loss(M, q₁, data, naive_log_q_data_r₁)
    predicted_curvature_corrected_distances_r₁[rank] = curvature_corrected_loss(M, q₁, data, curvature_corrected_log_q_data_r₁)
    true_curvature_corrected_distances_r₁[rank] = exact_loss(M, q₁, data, curvature_corrected_log_q_data_r₁)
    exact_distances_r₁[rank] = exact_loss(M, q₁, data, exact_log_q_data_r₁)

    ## basepoint q₂
    naive_log_q_data_r₂ = [sum([nR_q₂[rank][i] * nU₂[rank][k,i] for i in 1:rank]) for k in 1:n]
    curvature_corrected_log_q_data_r₂ = [sum([ccR_q₂[rank][i] * ccU₂[rank][k,i] for i in 1:rank]) for k in 1:n]
    exact_log_q_data_r₂ = [sum([eR_q₂[rank][i] * eU₂[rank][k,i] for i in 1:rank]) for k in 1:n]
    
    # exponentiate back
    naive_data_r₂ = exp.(Ref(M), Ref(q₂), naive_log_q_data_r₂)
    curvature_corrected_data_r₂ = exp.(Ref(M), Ref(q₂), curvature_corrected_log_q_data_r₂)
    exact_data_r₂ = exp.(Ref(M), Ref(q₂), exact_log_q_data_r₂)


    # compute relative tangent space error
    naive_tangent_distances_r₂[rank] = sum(norm.(Ref(M), Ref(q₂),  log_q_data₂ - naive_log_q_data_r₂).^2) / ref_distance₂
    curvature_corrected_tangent_distances_r₂[rank] = sum(norm.(Ref(M), Ref(q₂),  log_q_data₂ - curvature_corrected_log_q_data_r₂).^2) / ref_distance₂
    exact_tangent_distances_r₂[rank] = sum(norm.(Ref(M), Ref(q₂),  log_q_data₂ - exact_log_q_data_r₂).^2) / ref_distance₂


    # compute relative manifold error
    predicted_naive_distances_r₂[rank] = curvature_corrected_loss(M, q₂, data, naive_log_q_data_r₂)
    true_naive_distances_r₂[rank] = exact_loss(M, q₂, data, naive_log_q_data_r₂)
    predicted_curvature_corrected_distances_r₂[rank] = curvature_corrected_loss(M, q₂, data, curvature_corrected_log_q_data_r₂)
    true_curvature_corrected_distances_r₂[rank] = exact_loss(M, q₂, data, curvature_corrected_log_q_data_r₂)
    exact_distances_r₂[rank] = exact_loss(M, q₂, data, exact_log_q_data_r₂)
end

In [12]:
# basepoint q₁
plot(1:d-1, [β(κ₁)^2 .* naive_tangent_distances_r₁[1:end-1], true_naive_distances_r₁[1:end-1], true_curvature_corrected_distances_r₁[1:end-1], exact_distances_r₁[1:end-1]], label = ["zero-δ lower bound" "tSVD" "CC-tSVD (proposed)" "MC-tSVD"], xlims=(1,d-1),xaxis=("approximation rank"), yaxis=(L"$\varepsilon_{rel}$"))
savefig("results/artificial1D_errors_by_rank_1.svg")
plot(1:d-1, [β(κ₁)^2 .* naive_tangent_distances_r₁[1:end-1] .+ 1e-4, true_naive_distances_r₁[1:end-1] .+ 1e-4, true_curvature_corrected_distances_r₁[1:end-1] .+ 1e-4, exact_distances_r₁[1:end-1] .+ 1e-4], label = ["zero-δ lower bound" "tSVD" "CC-tSVD (proposed)" "MC-tSVD"], ylims=(1e-4,1), xlims=(1,d-1), xaxis=("approximation rank"), yaxis=(L"$\varepsilon_{rel}$", :log), legend=:bottomleft)
savefig("results/artificial1D_logerrors_by_rank_1.svg")
for i in 1:d-1
    if i == 1
        plot(1:length(eCosts₁[1]), eCosts₁[1], label = "rank 1", ylims=(1e-4,1), yaxis=(L"$\varepsilon_{rel}$", :log))
    else
        plot!(1:length(eCosts₁[i]), eCosts₁[i], label = "rank $(i)", ylims=(1e-4,1), yaxis=(L"$\varepsilon_{rel}$", :log))
    end
end
savefig("results/artificial1D_exact_iterate_loss_1.svg")

# basepoint q₂
plot(1:d-1, [β(κ₂)^2 .* naive_tangent_distances_r₂[1:end-1], true_naive_distances_r₂[1:end-1], true_curvature_corrected_distances_r₂[1:end-1], exact_distances_r₂[1:end-1]], label = ["zero-δ lower bound" "tSVD" "CC-tSVD (proposed)" "MC-tSVD"], xlims=(1,d-1),xaxis=("approximation rank"), yaxis=(L"$\varepsilon_{rel}$"))
savefig("results/artificial1D_errors_by_rank_2.svg")
plot(1:d-1, [β(κ₂)^2 .* naive_tangent_distances_r₂[1:end-1] .+ 1e-4, true_naive_distances_r₂[1:end-1] .+ 1e-4, true_curvature_corrected_distances_r₂[1:end-1] .+ 1e-4, exact_distances_r₂[1:end-1] .+ 1e-4], label = ["zero-δ lower bound" "tSVD" "CC-tSVD (proposed)" "MC-tSVD"], ylims=(1e-4,1), xlims=(1,d-1), xaxis=("approximation rank"), yaxis=(L"$\varepsilon_{rel}$", :log), legend=:bottomleft)
savefig("results/artificial1D_logerrors_by_rank_2.svg")
for i in 1:d-1
    if i == 1
        plot(1:length(eCosts₂[1]), eCosts₂[1], label = "rank 1", ylims=(1e-4,1), yaxis=(L"$\varepsilon_{rel}$", :log))
    else
        plot!(1:length(eCosts₂[i]), eCosts₂[i], label = "rank $(i)", ylims=(1e-4,1), yaxis=(L"$\varepsilon_{rel}$", :log))
    end
end
savefig("results/artificial1D_exact_iterate_loss_2.svg")

"/Users/wdiepeveen/Documents/PhD/Projects/8 - Manifold-valued tensor decomposition/src/manifold-valued-tensors/experiments/1D/S2/results/artificial1D_exact_iterate_loss_2.svg"

In [13]:
# basepoint q₁
plot(1:d-1, (predicted_curvature_corrected_distances_r₁[1:end-1] .- true_curvature_corrected_distances_r₁[1:end-1] .+ 1e-16) ./ (curvature_corrected_tangent_distances_r₁[1:end-1] .* sqrt.(curvature_corrected_tangent_distances_r₁[1:end-1] .* ref_distance₁) .+ 1e-16), label=("CC-tSVD (proposed)"), xlims=(1,d-1),xaxis=("approximation rank"), yaxis=(L"$\delta_{rel}$"), color=3)
savefig("results/artificial1D_discrepancy_by_rank_1.svg")

# basepoint q₂
plot(1:d-1, (predicted_curvature_corrected_distances_r₂[1:end-1] .- true_curvature_corrected_distances_r₂[1:end-1] .+ 1e-16) ./ (curvature_corrected_tangent_distances_r₂[1:end-1] .* sqrt.(curvature_corrected_tangent_distances_r₂[1:end-1] .* ref_distance₂) .+ 1e-16), label=("CC-tSVD (proposed)"), xlims=(1,d-1),xaxis=("approximation rank"), yaxis=(L"$\delta_{rel}$"), color=3)
savefig("results/artificial1D_discrepancy_by_rank_2.svg")

"/Users/wdiepeveen/Documents/PhD/Projects/8 - Manifold-valued tensor decomposition/src/manifold-valued-tensors/experiments/1D/S2/results/artificial1D_discrepancy_by_rank_2.svg"

### Benchmarking ###

1) benchmark for q₁

In [14]:
@benchmark naive_low_rank_approximation(M, q₁, data, 2)

BenchmarkTools.Trial: 6805 samples with 1 evaluation.
 Range (min … max):  539.625 μs …   8.302 ms  ┊ GC (min … max): 0.00% … 87.56%
 Time  (median):     638.764 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   728.551 μs ± 476.411 μs  ┊ GC (mean ± σ):  4.27% ±  6.12%

        ██▄                                                      
  ▁▁▁▁▁▃████▆▅▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  540 μs           Histogram: frequency by time         1.19 ms <

 Memory estimate: 333.58 KiB, allocs estimate: 522.

In [15]:
@benchmark curvature_corrected_low_rank_approximation(M, q₁, data, 2) 

BenchmarkTools.Trial: 2273 samples with 1 evaluation.
 Range (min … max):  1.639 ms … 11.247 ms  ┊ GC (min … max): 0.00% … 81.79%
 Time  (median):     1.902 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   2.189 ms ±  1.044 ms  ┊ GC (mean ± σ):  6.49% ± 10.87%

  ▂█▆▄▄▃▄▃▃▂                                                  
  ███████████▇▇▆▆▄▁▄▁▁▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▄▅▅▅▃▃▅▅▅ █
  1.64 ms      Histogram: log(frequency) by time     8.64 ms <

 Memory estimate: 1.48 MiB, allocs estimate: 4616.

In [16]:
@benchmark exact_low_rank_approximation(M, q₁, data, 2; stepsize=1/64, max_iter=1) 

BenchmarkTools.Trial: 666 samples with 1 evaluation.
 Range (min … max):  5.165 ms … 38.281 ms  ┊ GC (min … max):  0.00% …  0.00%
 Time  (median):     6.223 ms              ┊ GC (median):     0.00%
 Time  (mean ± σ):   7.490 ms ±  3.174 ms  ┊ GC (mean ± σ):  10.43% ± 16.29%

  ▁▆█▇▇▆▅▃▃▂▂▁▁▁▁   ▁                                         
  ███████████████▇▇██▇▆█▅█▄▆▇▄▄▆▅▁▁▄▄▄▅▅▆▇▆▆▁▆▆▁▅█▆▇▇▁▆▇▁▁▄▆ █
  5.17 ms      Histogram: log(frequency) by time     17.6 ms <

 Memory estimate: 6.38 MiB, allocs estimate: 19985.

In [17]:
@benchmark exact_low_rank_approximation(M, q₁, data, 2; stepsize=1/64, max_iter=max_iter)  

BenchmarkTools.Trial: 13 samples with 1 evaluation.
 Range (min … max):  351.732 ms … 514.567 ms  ┊ GC (min … max): 13.76% … 12.60%
 Time  (median):     415.694 ms               ┊ GC (median):    12.64%
 Time  (mean ± σ):   409.835 ms ±  40.940 ms  ┊ GC (mean ± σ):  12.73% ±  1.30%

  ▁  ▁   ▁    ▁   ▁    ▁ ▁▁▁ █   ▁                            ▁  
  █▁▁█▁▁▁█▁▁▁▁█▁▁▁█▁▁▁▁█▁███▁█▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  352 ms           Histogram: frequency by time          515 ms <

 Memory estimate: 377.43 MiB, allocs estimate: 1089963.

In [18]:
nT₁ = []
nΣ₁ = []

for i in 1:d
    nbm₁ = @benchmark naive_low_rank_approximation(M, q₁, data, $i)
    push!(nT₁, mean(nbm₁).time / 1e9)
    push!(nΣ₁, std(nbm₁).time / 1e9)
end

In [19]:
ccT₁ = []
ccΣ₁ = []

for i in 1:d
    ccbm₁ = @benchmark curvature_corrected_low_rank_approximation(M, q₁, data, $i)
    push!(ccT₁, mean(ccbm₁).time / 1e9)
    push!(ccΣ₁, std(ccbm₁).time / 1e9)
end

In [20]:
eT1₁ = []
eΣ1₁ = []

for i in 1:d
    ebm1₁ = @benchmark exact_low_rank_approximation(M, q₁, data, $i; stepsize=1/64, max_iter=1)
    push!(eT1₁, mean(ebm1₁).time / 1e9)
    push!(eΣ1₁, std(ebm1₁).time / 1e9)
end

In [21]:
eT₁ = []
eΣ₁ = []

for i in 1:d
    ebm₁ = @benchmark exact_low_rank_approximation(M, q₁, data, $i; stepsize=1/64, max_iter=max_iter)
    push!(eT₁, mean(ebm₁).time / 1e9)
    push!(eΣ₁, std(ebm₁).time / 1e9)
end

In [22]:
# methods above each other and results per rank in colums
println("tSVD" * prod([" & " * raw"$" * "$(Float16(nT₁[i]))" * raw"$" for i in 1:d-1]) * raw"\\ ")
println("CC-tSVD (proposed)" * prod([" & " * raw"$" * "$(Float16(ccT₁[i]))" * raw"$" for i in 1:d-1]) * raw"\\ ")
println("MC-tSVD (1 iteration)" * prod([" & " * raw"$" * "$(Float16(eT1₁[i]))" * raw"$" for i in 1:d-1]) * raw"\\ ")
println("MC-tSVD" * prod([" & " * raw"$" * "$(Float16(eT₁[i]))" * raw"$" for i in 1:d-1]) * raw"\\ ")

tSVD & $0.0008435$ & $0.0007377$ & $0.0008345$ & $0.001223$ & $0.00087$ & $0.00098$\\ 
CC-tSVD (proposed) & $0.00193$ & $0.002363$ & $0.002367$ & $0.002722$ & $0.002932$ & $0.003197$\\ 
MC-tSVD (1 iteration) & $0.00629$ & $0.006916$ & $0.00837$ & $0.00856$ & $0.008385$\\ 
MC-tSVD & $0.3813$ & $0.3687$ & $0.3984$ & $0.4485$ & $0.508$\\ 


2) benchmark for q₂

In [23]:
@benchmark naive_low_rank_approximation(M, q₂, data, 2)

BenchmarkTools.Trial: 6678 samples with 1 evaluation.
 Range (min … max):  546.513 μs …   9.118 ms  ┊ GC (min … max): 0.00% … 88.30%
 Time  (median):     647.202 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   741.945 μs ± 509.255 μs  ┊ GC (mean ± σ):  4.38% ±  6.12%

        █▇▂                                                      
  ▁▂▁▁▁▃███▇▅▄▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  547 μs           Histogram: frequency by time         1.21 ms <

 Memory estimate: 333.58 KiB, allocs estimate: 522.

In [24]:
@benchmark curvature_corrected_low_rank_approximation(M, q₂, data, 2) 

BenchmarkTools.Trial: 2142 samples with 1 evaluation.
 Range (min … max):  1.651 ms … 12.292 ms  ┊ GC (min … max): 0.00% … 84.88%
 Time  (median):     2.022 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   2.321 ms ±  1.078 ms  ┊ GC (mean ± σ):  6.19% ± 10.82%

   █                                                          
  ▃██▆▅▅▄▄▄▃▃▃▂▂▂▂▂▁▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂▂▂▂▂▂ ▂
  1.65 ms        Histogram: frequency by time        8.87 ms <

 Memory estimate: 1.48 MiB, allocs estimate: 4616.

In [25]:
@benchmark exact_low_rank_approximation(M, q₂, data, 2; stepsize=1/16, max_iter=1) 

BenchmarkTools.Trial: 695 samples with 1 evaluation.
 Range (min … max):  5.169 ms … 18.752 ms  ┊ GC (min … max):  0.00% … 65.01%
 Time  (median):     6.281 ms              ┊ GC (median):     0.00%
 Time  (mean ± σ):   7.178 ms ±  2.513 ms  ┊ GC (mean ± σ):  10.29% ± 16.13%

   ▁▂█▅▂▁                                                     
  ▃██████▇▆▅▅▅▄▃▄▃▃▃▃▂▃▂▂▂▁▁▁▁▁▁▁▂▂▂▁▂▂▂▂▂▁▃▂▂▂▂▂▃▃▂▂▂▂▂▂▁▂▂ ▃
  5.17 ms        Histogram: frequency by time        16.6 ms <

 Memory estimate: 6.38 MiB, allocs estimate: 19985.

In [26]:
@benchmark exact_low_rank_approximation(M, q₂, data, 2; stepsize=1/16, max_iter=max_iter)  

BenchmarkTools.Trial: 4 samples with 1 evaluation.
 Range (min … max):  1.458 s …   1.632 s  ┊ GC (min … max): 14.29% … 12.87%
 Time  (median):     1.474 s              ┊ GC (median):    14.19%
 Time  (mean ± σ):   1.509 s ± 81.794 ms  ┊ GC (mean ± σ):  13.73% ±  0.68%

  █   █ █                                                 █  
  █▁▁▁█▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  1.46 s         Histogram: frequency by time        1.63 s <

 Memory estimate: 1.49 GiB, allocs estimate: 4402494.

In [27]:
nT₂ = []
nΣ₂ = []

for i in 1:d
    nbm₂ = @benchmark naive_low_rank_approximation(M, q₂, data, $i)
    push!(nT₂, mean(nbm₂).time / 1e9)
    push!(nΣ₂, std(nbm₂).time / 1e9)
end

In [28]:
ccT₂ = []
ccΣ₂ = []

for i in 1:d
    ccbm₂ = @benchmark curvature_corrected_low_rank_approximation(M, q₂, data, $i)
    push!(ccT₂, mean(ccbm₂).time / 1e9)
    push!(ccΣ₂, std(ccbm₂).time / 1e9)
end

In [29]:
eT1₂ = []
eΣ1₂ = []

for i in 1:d
    ebm1₂ = @benchmark exact_low_rank_approximation(M, q₂, data, $i; stepsize=1/16, max_iter=1)
    push!(eT1₂, mean(ebm1₂).time / 1e9)
    push!(eΣ1₂, std(ebm1₂).time / 1e9)
end

In [30]:
eT₂ = []
eΣ₂ = []

for i in 1:d
    ebm₂ = @benchmark exact_low_rank_approximation(M, q₂, data, $i; stepsize=1/16, max_iter=max_iter)
    push!(eT₂, mean(ebm₂).time / 1e9)
    push!(eΣ₂, std(ebm₂).time / 1e9)
end

In [32]:
# methods above each other and results per rank in colums
println("tSVD" * prod([" & " * raw"$" * "$(Float16(nT₂[i]))" * raw"$" for i in 1:d-1]) * raw"\\ ")
println("CC-tSVD (proposed)" * prod([" & " * raw"$" * "$(Float16(ccT₂[i]))" * raw"$" for i in 1:d-1]) * raw"\\ ")
println("MC-tSVD (1 iteration)" * prod([" & " * raw"$" * "$(Float16(eT1₂[i]))" * raw"$" for i in 1:d-1]) * raw"\\ ")
println("MC-tSVD" * prod([" & " * raw"$" * "$(Float16(eT₂[i]))" * raw"$" for i in 1:d-1]) * raw"\\ ")

tSVD & $0.0008183$ & $0.0008283$ & $0.0009403$ & $0.0008883$ & $0.000927$\\ 
CC-tSVD (proposed) & $0.001951$ & $0.002422$ & $0.002525$ & $0.003294$ & $0.003426$\\ 
MC-tSVD (1 iteration) & $0.00653$ & $0.00752$ & $0.00781$ & $0.008286$ & $0.00883$\\ 
MC-tSVD & $0.2112$ & $1.524$ & $1.911$ & $2.178$ & $1.934$\\ 
